# CSCE 629 Analysis of Algorithms 


# Project 1

### Details about running the code : 

### Load the libraries

In [1]:
import pandas as pd
import numpy as np
import pickle
import sys

### Load the dictionary files

In [2]:
example_instances = pickle.load(open('examples_of_instances', 'rb'))
example_solutions = pickle.load(open('examples_of_solutions', 'rb'))
examples_of_small_instances = pickle.load(open('examples_of_small_instances', 'rb'))
examples_of_medium_instances = pickle.load(open('examples_of_medium_instances', 'rb'))
examples_of_large_instances = pickle.load(open('examples_of_large_instances', 'rb'))

### Calculate Error Matrix variables

In [3]:
def preprocess_arrays(n_list,x_list,y_list):
    #let's find cummulative sum for all the points error calculation variables
    
    #consider points from 0 to n_list-1 , sum[0] means sum till the point 0
    
    
    #create an X sum array with 0's
    X_sum = np.zeros(n_list)
    #create a Y sum array with 0's 
    Y_sum = np.zeros(n_list)
    #create an array for sigma X**2 with 0's
    X_pow_2_sum = np.zeros(n_list)
    #create an array for sigma Y**2 with 0's
    Y_pow_2_sum = np.zeros(n_list)
    #XY dot product _ sum
    XY_dot = np.zeros(n_list)
    
    #let's calculate the cummulative sum for all these variables in a single for loop
    
    # let's calculate it for 0
    X_sum[0] = x_list[0]
    Y_sum[0] = y_list[0]
    X_pow_2_sum[0] = x_list[0]*x_list[0]
    Y_pow_2_sum[0] = y_list[0]*y_list[0]
    XY_dot[0] = x_list[0]*y_list[0]
 
    #calc cummulative sum for remaining points
    for i in range(0,n_list):
        X_sum[i] = X_sum[i-1]+x_list[i]
        Y_sum[i] = Y_sum[i-1]+y_list[i]
        X_pow_2_sum[i] = X_pow_2_sum[i-1] + (x_list[i]*x_list[i])
        Y_pow_2_sum[i] = Y_pow_2_sum[i-1] + (y_list[i]*y_list[i])
        XY_dot[i] = XY_dot[i-1] + (x_list[i]*y_list[i])
        
    return X_sum,Y_sum,X_pow_2_sum,Y_pow_2_sum,XY_dot
        

### Calculate Error Matrix

In [4]:
def linear_regressor_error_matrix(n_list,x_list,y_list,c_list):
    
    #create an error matrix for the given points 
    
    #error_matrix[i][j] => error from point i to j segment
    #i from 0 to n_list-1 and j from 0 to n_list-1
    
    error_matrix = np.zeros((n_list, n_list))
    
    # let's the cummulative sum parameters
    
    X_sum,Y_sum,X_pow_2_sum,Y_pow_2_sum,XY_dot = preprocess_arrays(n_list,x_list,y_list)
    
    for i in range(0,n_list):
        for j in range(i+1,n_list):
            
            #if i and j are adjacent point, no need to calculate error, as it is going to be 0, just add penalty
            if(j-i == 1):
                error_matrix[i][j] = c_list
            
            #let's write the if condition where it checks if 0th point is involved
            #here we can directly use cummulative sum till jth point
            elif(i==0):
                #total points n
                n = j+1
                #calc individual params
                X_sum_ij = X_sum[j]
                Y_sum_ij = Y_sum[j]
                X_pow_2_sum_ij = X_pow_2_sum[j]
                Y_pow_2_sum_ij = Y_pow_2_sum[j]
                XY_dot_ij = XY_dot[j]
                a=((n*XY_dot_ij)-(X_sum_ij*Y_sum_ij))/((n*X_pow_2_sum_ij)-((X_sum_ij)*(X_sum_ij)))
                b=(Y_sum_ij-a*X_sum_ij)/(n)
                error_matrix[i][j]=abs((Y_pow_2_sum_ij)+((a*a)*X_pow_2_sum_ij)+(n*(b*b))+(2*a*b*X_sum_ij)-(2*a*XY_dot_ij)-(2*b*Y_sum_ij))+c_list
            else:
                n = j-i+1
                #calc individual params
                X_sum_ij = X_sum[j] - X_sum[i-1]
                Y_sum_ij = Y_sum[j] - Y_sum[i-1]
                X_pow_2_sum_ij = X_pow_2_sum[j] - X_pow_2_sum[i-1]
                Y_pow_2_sum_ij = Y_pow_2_sum[j] - Y_pow_2_sum[i-1]
                XY_dot_ij = XY_dot[j] - XY_dot[i-1]
                a=((n*XY_dot_ij)-(X_sum_ij*Y_sum_ij))/((n*X_pow_2_sum_ij)-((X_sum_ij)*(X_sum_ij)))
                b=(Y_sum_ij-a*X_sum_ij)/(n)
                error_matrix[i][j]=abs((Y_pow_2_sum_ij)+((a*a)*X_pow_2_sum_ij)+(n*(b*b))+(2*a*b*X_sum_ij)-(2*a*XY_dot_ij)-(2*b*Y_sum_ij))+c_list

    return error_matrix
    

### Calculate the result for a given data set record

In [5]:
def piecewise_linear_segmentation(record):
    n_list = record[0]
    x_list = record[1]
    y_list = record[2]
    c_list = record[3]
    
    #calculate and store errors for the possible ranges
    errors = linear_regressor_error_matrix(n_list,x_list,y_list,c_list)
            
    # Lets Calculate (k_list,last_points_list, OPT_list)
    # k_list = number of segments
    # last_points_list = list of last points in each segment
    # OPT_list is the cost function value 
    
    # Lets create a list dp to store the cost till a point (this stores the OPT_list value in dp[n_list-1])
    # dp[i] means cost till ith point

    dp = [0 for i in range(n_list)]
    dp[1] = errors[0][1]
    
    # Lets track K value for every i value in DP (this stores the last_points_list in K[n_list-1])
    K = [[] for i in range(n_list) ]
    K[1].append(1)
    
    # run for ith point cost in dp
    for i in range(2,n_list):

        #store min for this dp point 
        min_val = sys.maxsize
        K[i].append(i)
        temp_list = []
        #number of steps we are going back to check the cost
        for k in range(2,i+1):
            if ((i-k) > 0):
                temp_val = dp[i-k] + errors[i-k+1][i]
                # now lets check if temp_val is less than min_val; if yes lets save it's break points
                if(temp_val < min_val):
                    min_val = temp_val
                    temp_list = K[i-k]
    #             min_val = min(temp_val,min_val)
            else:
                temp_val = errors[0][i]
                if(temp_val < min_val):
                    min_val = temp_val
                    temp_list = []
    #             min_val = min(temp_val,min_val)
    #             temp_list = []

        dp[i] = min_val
        K[i] = K[i]+temp_list 
        
#     print("K list is")
#     print(K[n_list-1])
#     print("Optimum value is")
#     print(dp[n_list-1])
#     print("Segments count is")
#     print(len(K[n_list-1]))
    result = {}
    result['k_list'] = len(K[n_list-1])
    result['last_points_list'] = K[n_list-1]
    result['OPT_list'] = dp[n_list-1]
    return result

### Main function to pass the dataset

In [6]:
def piecewise_dict(data_set):
    result_dict = {'k_list':[],'last_points_list':[],'OPT_list':[]}
    len_data = len(data_set['n_list'])
    temp_data = []
    for i in range(0,len_data):
        temp_data = []
        temp_data.append(data_set['n_list'][i])
        temp_data.append(data_set['x_list'][i])
        temp_data.append(data_set['y_list'][i])
        temp_data.append(data_set['C_list'][i])
        temp_res = piecewise_linear_segmentation(temp_data)
        result_dict['k_list'].append(temp_res['k_list'])
        result_dict['last_points_list'].append(temp_res['last_points_list'])
        result_dict['OPT_list'].append(temp_res['OPT_list'])
        
    return result_dict

### Calculate result for all the given instances

In [7]:
piecewise_dict(example_instances)

{'k_list': [3, 5, 7, 5, 5, 6, 5, 6, 5, 5],
 'last_points_list': [[93, 44, 31],
  [95, 75, 49, 34, 17],
  [76, 72, 70, 62, 49, 45, 21],
  [97, 67, 58, 35, 14],
  [104, 80, 54, 32, 21],
  [129, 119, 100, 71, 46, 16],
  [88, 65, 52, 38, 9],
  [122, 101, 72, 69, 46, 24],
  [94, 69, 55, 30, 10],
  [97, 73, 47, 35, 11]],
 'OPT_list': [9955.201272374872,
  12127.705100827989,
  9763.325183154073,
  10943.868071846016,
  124.31733328050666,
  165.22998494411172,
  124.36266382539344,
  175.08797571155932,
  5.000000001133344,
  5.000000000116547]}

In [8]:
small = piecewise_dict(examples_of_small_instances)
small

{'k_list': [20,
  24,
  24,
  20,
  19,
  20,
  24,
  19,
  20,
  17,
  21,
  24,
  23,
  20,
  20,
  19,
  18,
  22,
  24,
  17,
  21,
  21,
  21,
  21,
  21,
  22,
  22,
  21,
  21,
  20,
  21,
  20,
  20,
  21,
  17,
  21,
  21,
  20,
  20,
  19,
  19,
  19,
  20,
  20,
  20,
  19,
  20,
  19,
  20,
  20,
  12,
  10,
  12,
  10,
  7,
  9,
  10,
  13,
  8,
  11,
  12,
  12,
  12,
  7,
  12,
  11,
  13,
  10,
  13,
  10,
  10,
  10,
  10,
  15,
  11,
  11,
  10,
  10,
  12,
  10,
  10,
  11,
  13,
  11,
  13,
  9,
  10,
  8,
  11,
  10,
  9,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10],
 'last_points_list': [[361,
   351,
   338,
   316,
   297,
   287,
   270,
   261,
   256,
   246,
   226,
   203,
   177,
   155,
   137,
   121,
   92,
   69,
   46,
   30],
  [404,
   399,
   397,
   388,
   375,
   359,
   351,
   332,
   330,
   321,
   317,
   306,
   284,
   267,
   246,
   231,
   213,
   185,
   166,
   143,
   135,
   95,
   33,
   30],
  [389,
   374,
   339,
   32

In [9]:
with open('small.pkl', 'wb') as f:
    pickle.dump(small, f)

In [10]:
read_small = pickle.load(open('small.pkl', 'rb'))
read_small

{'k_list': [20,
  24,
  24,
  20,
  19,
  20,
  24,
  19,
  20,
  17,
  21,
  24,
  23,
  20,
  20,
  19,
  18,
  22,
  24,
  17,
  21,
  21,
  21,
  21,
  21,
  22,
  22,
  21,
  21,
  20,
  21,
  20,
  20,
  21,
  17,
  21,
  21,
  20,
  20,
  19,
  19,
  19,
  20,
  20,
  20,
  19,
  20,
  19,
  20,
  20,
  12,
  10,
  12,
  10,
  7,
  9,
  10,
  13,
  8,
  11,
  12,
  12,
  12,
  7,
  12,
  11,
  13,
  10,
  13,
  10,
  10,
  10,
  10,
  15,
  11,
  11,
  10,
  10,
  12,
  10,
  10,
  11,
  13,
  11,
  13,
  9,
  10,
  8,
  11,
  10,
  9,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10],
 'last_points_list': [[361,
   351,
   338,
   316,
   297,
   287,
   270,
   261,
   256,
   246,
   226,
   203,
   177,
   155,
   137,
   121,
   92,
   69,
   46,
   30],
  [404,
   399,
   397,
   388,
   375,
   359,
   351,
   332,
   330,
   321,
   317,
   306,
   284,
   267,
   246,
   231,
   213,
   185,
   166,
   143,
   135,
   95,
   33,
   30],
  [389,
   374,
   339,
   32

In [11]:
med = piecewise_dict(examples_of_medium_instances)
med

{'k_list': [76,
  81,
  86,
  83,
  83,
  83,
  86,
  92,
  80,
  76,
  85,
  86,
  86,
  94,
  79,
  93,
  87,
  94,
  88,
  86,
  79,
  79,
  78,
  80,
  78,
  80,
  77,
  79,
  78,
  76,
  37,
  47,
  43,
  47,
  43,
  38,
  40,
  46,
  42,
  43,
  47,
  37,
  42,
  44,
  41,
  44,
  36,
  42,
  38,
  40,
  39,
  42,
  42,
  36,
  46,
  41,
  45,
  41,
  46,
  48,
  42,
  43,
  44,
  47,
  44,
  45,
  45,
  44,
  44,
  44,
  40,
  39,
  40,
  40,
  38,
  40,
  40,
  40,
  40,
  39],
 'last_points_list': [[1609,
   1593,
   1569,
   1560,
   1537,
   1507,
   1493,
   1483,
   1460,
   1450,
   1436,
   1422,
   1409,
   1380,
   1372,
   1343,
   1331,
   1310,
   1301,
   1296,
   1273,
   1255,
   1228,
   1209,
   1185,
   1172,
   1151,
   1143,
   1133,
   1120,
   1114,
   1092,
   1079,
   1076,
   1050,
   1034,
   985,
   956,
   942,
   916,
   905,
   892,
   846,
   843,
   818,
   807,
   797,
   771,
   720,
   712,
   701,
   676,
   648,
   620,
   603,
   576,
   54

In [12]:
with open('med.pkl', 'wb') as f:
    pickle.dump(med, f)

In [13]:
read_med = pickle.load(open('med.pkl', 'rb'))
read_med

{'k_list': [76,
  81,
  86,
  83,
  83,
  83,
  86,
  92,
  80,
  76,
  85,
  86,
  86,
  94,
  79,
  93,
  87,
  94,
  88,
  86,
  79,
  79,
  78,
  80,
  78,
  80,
  77,
  79,
  78,
  76,
  37,
  47,
  43,
  47,
  43,
  38,
  40,
  46,
  42,
  43,
  47,
  37,
  42,
  44,
  41,
  44,
  36,
  42,
  38,
  40,
  39,
  42,
  42,
  36,
  46,
  41,
  45,
  41,
  46,
  48,
  42,
  43,
  44,
  47,
  44,
  45,
  45,
  44,
  44,
  44,
  40,
  39,
  40,
  40,
  38,
  40,
  40,
  40,
  40,
  39],
 'last_points_list': [[1609,
   1593,
   1569,
   1560,
   1537,
   1507,
   1493,
   1483,
   1460,
   1450,
   1436,
   1422,
   1409,
   1380,
   1372,
   1343,
   1331,
   1310,
   1301,
   1296,
   1273,
   1255,
   1228,
   1209,
   1185,
   1172,
   1151,
   1143,
   1133,
   1120,
   1114,
   1092,
   1079,
   1076,
   1050,
   1034,
   985,
   956,
   942,
   916,
   905,
   892,
   846,
   843,
   818,
   807,
   797,
   771,
   720,
   712,
   701,
   676,
   648,
   620,
   603,
   576,
   54

In [14]:
large = piecewise_dict(examples_of_large_instances)
large

{'k_list': [357, 147, 163, 181, 168, 159, 157],
 'last_points_list': [[6361,
   6342,
   6321,
   6304,
   6276,
   6246,
   6235,
   6233,
   6218,
   6207,
   6186,
   6163,
   6151,
   6148,
   6121,
   6109,
   6094,
   6085,
   6071,
   6060,
   6042,
   6027,
   6024,
   6003,
   5990,
   5979,
   5973,
   5948,
   5922,
   5905,
   5889,
   5876,
   5860,
   5847,
   5836,
   5807,
   5784,
   5758,
   5755,
   5733,
   5719,
   5696,
   5676,
   5651,
   5622,
   5594,
   5578,
   5549,
   5547,
   5525,
   5499,
   5469,
   5444,
   5434,
   5424,
   5413,
   5403,
   5385,
   5371,
   5341,
   5319,
   5308,
   5297,
   5278,
   5266,
   5253,
   5242,
   5214,
   5190,
   5175,
   5156,
   5127,
   5102,
   5089,
   5084,
   5072,
   5058,
   5031,
   5002,
   4989,
   4959,
   4954,
   4930,
   4908,
   4888,
   4863,
   4859,
   4835,
   4823,
   4801,
   4781,
   4762,
   4756,
   4731,
   4709,
   4689,
   4661,
   4638,
   4632,
   4622,
   4617,
   4590,
   4586,
   45

In [15]:
with open('large.pkl', 'wb') as f:
    pickle.dump(large, f)

In [16]:
read_large = pickle.load(open('large.pkl', 'rb'))
read_large

{'k_list': [357, 147, 163, 181, 168, 159, 157],
 'last_points_list': [[6361,
   6342,
   6321,
   6304,
   6276,
   6246,
   6235,
   6233,
   6218,
   6207,
   6186,
   6163,
   6151,
   6148,
   6121,
   6109,
   6094,
   6085,
   6071,
   6060,
   6042,
   6027,
   6024,
   6003,
   5990,
   5979,
   5973,
   5948,
   5922,
   5905,
   5889,
   5876,
   5860,
   5847,
   5836,
   5807,
   5784,
   5758,
   5755,
   5733,
   5719,
   5696,
   5676,
   5651,
   5622,
   5594,
   5578,
   5549,
   5547,
   5525,
   5499,
   5469,
   5444,
   5434,
   5424,
   5413,
   5403,
   5385,
   5371,
   5341,
   5319,
   5308,
   5297,
   5278,
   5266,
   5253,
   5242,
   5214,
   5190,
   5175,
   5156,
   5127,
   5102,
   5089,
   5084,
   5072,
   5058,
   5031,
   5002,
   4989,
   4959,
   4954,
   4930,
   4908,
   4888,
   4863,
   4859,
   4835,
   4823,
   4801,
   4781,
   4762,
   4756,
   4731,
   4709,
   4689,
   4661,
   4638,
   4632,
   4622,
   4617,
   4590,
   4586,
   45